In [1]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler
import numpy as np
import pandas as pd
import torch as torch

In [2]:
x_f = torch.rand(500, 456, 10)

In [3]:
x_f.shape

torch.Size([500, 456, 10])

In [4]:
idx = torch.add(torch.arange(365).unsqueeze(axis = 1), torch.arange(91).unsqueeze(axis = 0))

In [5]:
idx

tensor([[  0,   1,   2,  ...,  88,  89,  90],
        [  1,   2,   3,  ...,  89,  90,  91],
        [  2,   3,   4,  ...,  90,  91,  92],
        ...,
        [362, 363, 364,  ..., 450, 451, 452],
        [363, 364, 365,  ..., 451, 452, 453],
        [364, 365, 366,  ..., 452, 453, 454]])

In [6]:
expanded = x_f[:, idx, :] # expanded 

In [7]:
expanded.shape

torch.Size([500, 365, 91, 10])

In [8]:
expanded.view(500, 365, 910).shape # expand

torch.Size([500, 365, 910])

In [9]:
hf1 = nn.Linear(910, 30)(x_f[:, idx, :].view(-1, 365, 910))

In [10]:
hf1.shape

torch.Size([500, 365, 30])

In [11]:
x = torch.rand(500, 456, 50)

In [12]:
x_t = x[:, :365, 10:40]
x_f = x[:, :, 40:]

In [13]:
x_s = torch.rand(500, 10)

In [14]:
x_s.shape, x_t.shape, x_f.shape

(torch.Size([500, 10]), torch.Size([500, 365, 30]), torch.Size([500, 456, 10]))

In [15]:
s = nn.Linear(10, 30)(x_s)


In [16]:
c1 = F.pad(x_t.permute(0, 2, 1), (1, 0), "constant", 0)
c1 = nn.Conv1d(30, 30, 2, dilation = 1)(c1)
c2 = F.pad(c1, (2, 0), "constant", 0)
c2 = nn.Conv1d(30, 30, 2, dilation = 2)(c2)
c3 = F.pad(c2, (4, 0), "constant", 0)
c3 = nn.Conv1d(30, 30, 2,  dilation = 4)(c3)
c4 = F.pad(c3, (8, 0), "constant", 0)
c4 = nn.Conv1d(30, 30, 2, dilation = 8)(c4)
c5 = F.pad(c4, (16, 0), "constant", 0)
c5 = nn.Conv1d(30, 30, 2, dilation = 16)(c5)
c6 = F.pad(c5, (32, 0), "constant", 0)
c6 = nn.Conv1d(30, 30, 2, dilation = 32)(c6)


In [17]:
t = c6.permute(0, 2, 1)

In [18]:
s = s.unsqueeze(1).repeat(1, 365, 1)

In [19]:
t.shape

torch.Size([500, 365, 30])

In [20]:
encoding = torch.cat((s, t), axis = 2)

In [21]:
encoding.shape

torch.Size([500, 365, 60])

In [22]:
ht = torch.cat((hf1, encoding), axis = 2)

In [23]:
ht.shape

torch.Size([500, 365, 90])

In [24]:
nn.Linear(90, 1820)(ht).shape

torch.Size([500, 365, 1820])

In [25]:
ht1 = nn.Linear(90, 1820)(ht).view(-1, 365, 91, 20)

In [26]:
ht2 = nn.Linear(90, 100)(ht).unsqueeze(axis = 2).repeat(1, 1, 91, 1)

In [27]:
hf2 = F.relu(expanded)

In [28]:
expanded.shape, ht1.shape, ht2.shape

(torch.Size([500, 365, 91, 10]),
 torch.Size([500, 365, 91, 20]),
 torch.Size([500, 365, 91, 100]))

In [29]:
h = torch.cat((ht1, ht2, hf2), dim = -1)

In [30]:
class local_mlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(130, 50)
        self.l2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x  = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        
        return x

In [31]:
l = local_mlp()

In [32]:
h2 = l(h)

In [48]:
h2 = nn.Linear(10, 2)(h2)

In [52]:
y = F.leaky_relu(h2.contiguous().view(-1, h2.size(-2), h2.size(-1)))

In [68]:
y.view(-1, 365, 91, 2).view(500, 365, 182).shape

torch.Size([500, 365, 91, 2])

In [54]:
h.shape

torch.Size([500, 365, 91, 130])

In [66]:
h2.permute(0, 1, 3, 2).contiguous().view(500, 365, -1).shape

torch.Size([500, 365, 182])